<a href="https://colab.research.google.com/github/kanakesh2006/Practical-Deep-Learning-using-PyTorch/blob/main/PyTorch_Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def dy_dx(x):
  return 2*x

In [2]:
dy_dx(3)

6

In [3]:
import torch

In [4]:
x = torch.tensor(3.0, requires_grad=True)

In [5]:
y = x**2

In [6]:
x

tensor(3., requires_grad=True)

In [7]:
y

tensor(9., grad_fn=<PowBackward0>)

In [8]:
y.backward()

In [9]:
x.grad

tensor(6.)

In [10]:
import math

def dz_dx(x):
    return 2 * x * math.sin(x**2)

In [11]:
dz_dx(3)

2.4727109114505397

In [12]:
x = torch.tensor(3.0, requires_grad=True)

In [13]:
y = x ** 2

In [14]:
z = torch.sin(y)

In [15]:
u = 2 * x * z

In [16]:
x

tensor(3., requires_grad=True)

In [17]:
y

tensor(9., grad_fn=<PowBackward0>)

In [18]:
z

tensor(0.4121, grad_fn=<SinBackward0>)

In [19]:
u

tensor(2.4727, grad_fn=<MulBackward0>)

In [20]:
u.backward()

In [21]:
x.grad

tensor(-31.9765)

In [22]:
y.grad()

/tmp/ipython-input-301197645.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad()


TypeError: 'NoneType' object is not callable

u -> root node

x -> leaf node

### Implementing Autograd on Neural Network

#### Scratch code

In [23]:
import torch

#Inputs
x = torch.tensor(6.7) # Input feature
y = torch.tensor(0.0) # True label (binary)

w = torch.tensor(1.0) # Weight
b = torch.tensor(0.0) # Bias

In [24]:
# Binary Cross-Entropy Loss for Scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8 # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return - (target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [25]:
# Forward pass
z = w * x + b  #  weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [26]:
loss

tensor(6.7012)

In [27]:
# Derivatives:

dloss_dy_pred = (y_pred - y) / (y_pred * (1 - y_pred))

dy_pred_dz = y_pred * (1 - y_pred)

dz_dw = x
dz_db = 1

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [28]:
print(f"Manual Gradient of loss w.r.t weight (dl/dw) : {dL_dw}")
print(f"Manual Gradient of loss w.r.t weight (dl/db) : {dL_db}")

Manual Gradient of loss w.r.t weight (dl/dw) : 6.691762447357178
Manual Gradient of loss w.r.t weight (dl/db) : 0.998770534992218


#### Autograd code

In [29]:
#Inputs
x = torch.tensor(6.7) # Input feature
y = torch.tensor(0.0) # True label (binary)

w = torch.tensor(1.0, requires_grad=True) # Weight
b = torch.tensor(0.0, requires_grad=True) # Bias

In [30]:
w

tensor(1., requires_grad=True)

In [31]:
b

tensor(0., requires_grad=True)

In [32]:
z = w * x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [33]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [34]:
loss = binary_cross_entropy_loss(y_pred,y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [35]:
loss.backward()

In [36]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


In [37]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
x

tensor([1., 2., 3.], requires_grad=True)

In [38]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [39]:
y.backward()

In [40]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

### Clearing gradients

In [41]:
x = torch.tensor(2.0, requires_grad=True)

In [42]:
y = x**2

In [43]:
y.backward()

In [44]:
x.grad

tensor(4.)

In [45]:
x.grad.zero_()

tensor(0.)

### Disable gradient tracking

In [46]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [47]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [48]:
y.backward()

In [49]:
x.grad

tensor(4.)

In [50]:
# option 1 - .requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad()

In [51]:
# option 1 - .requires_grad_(False)
x.requires_grad_(False)

tensor(2.)

In [52]:
x

tensor(2.)

In [53]:
y = x**2

In [54]:
y

tensor(4.)

In [55]:
y.backwards()

AttributeError: 'Tensor' object has no attribute 'backwards'

In [56]:
# option 2 - detach()
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [57]:
z = x.detach()
z

tensor(2.)

In [58]:
y = x**2
y

tensor(4., grad_fn=<PowBackward0>)

In [59]:
y1 = z ** 2
y1

tensor(4.)

In [60]:
y.backward()

In [61]:
y1.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [62]:
# option 3 - torch.no_grad()
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [63]:
with torch.no_grad():
  y = x ** 2

In [64]:
y

tensor(4.)

In [65]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn